# CODA-database-sandbox

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

#### Date: 11 September 2024

# detect compute platform

In [2]:
import os
import socket
    
def get_platform():
    hostname = socket.gethostname()
    if "gadi" in hostname:  # Adjust this condition to fit your HPC's hostname or unique identifier
        return 'HPC',hostname
    else:
        return 'Laptop',hostname
[platform,hostname] = get_platform()
print('the platform we are working on is '+platform+' with hostname: '+hostname)

the platform we are working on is HPC with hostname: gadi-cpu-bdw-0760.gadi.nci.org.au


### import packages

In [3]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd

In [4]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34431,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:33447,Total threads: 4
Dashboard: /proxy/33871/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:36435,


In [5]:
if platform == 'HPC':
    data_path = '/g/data/es60/users/thomas_moore/CODA/2005/'
    write_path = '/g/data/es60/users/thomas_moore/CODA/parquet/'
else:
    data_path = '/Users/moo270/data/CARSv2/CODA/'
    write_path = '/Users/moo270/data/CARSv2/CODA/parquet/'

print(f"Using data path: {data_path}")
print(f"Using write path: {write_path}")

Using data path: /g/data/es60/users/thomas_moore/CODA/2005/
Using write path: /g/data/es60/users/thomas_moore/CODA/parquet/


# build PQ & CSV versions from NetCDF

In [ ]:
ds = xr.open_dataset(data_path+ "WOD2018_CODA_2005_pfl.nc",chunks="auto")

In [ ]:
ds

In [ ]:
ds = ds.set_coords(['WOD_id',
                    'origflagset',
                    'country',
                    'dataset',
                    'Access_no',
                    'dbase_orig',
                    'Project',
                    'WOD_cruise_identifier',
                    'Institute',
                    'Ocean_Vehicle',
                    'Temperature_Instrument',
                    'CODA_id'])

In [ ]:
ds

# make truncated version of ds

In [ ]:
ds_truncated = ds#.isel(cast=slice(0,10000))

In [ ]:
%%time
df = ds_truncated[['Temperature']].to_dataframe().reset_index()
df

# convert to dask dataframe

In [ ]:
%%time
ddf = dd.from_pandas(df, npartitions=500)

In [ ]:
ddf

# find columns with byte strings and convert ( is this needed when going from ds --> df --> ddf )

In [ ]:
ddf.head()

In [ ]:
%%time
ddf.info(memory_usage='deep')

In [ ]:
%%time
ddf.to_parquet(write_path+"2005_pfl_temp.pq",engine='pyarrow', compression='snappy')

In [ ]:
%%time
df.to_csv(write_path+"2005_pfl_temp.csv", index=False)

# load parquet data & filter

In [ ]:
%%time
temp_pfl_ddf = dd.read_parquet(write_path+"2005_pfl_temp.pq")
# Search for rows where 'column_name' matches a condition
result_parquet = temp_pfl_ddf[temp_pfl_ddf ['z'] >= 2000].compute()

In [ ]:
result_parquet

# load CSV data and filter

In [ ]:
%%time
temp_pfl_csv_ddf = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)  # 25 MB chunks
# Search for rows where 'column_name' matches a condition
result_csv = temp_pfl_csv_ddf[temp_pfl_csv_ddf['z'] >= 2000].compute()

# lazy load temperature

In [7]:
%%time
temp_pq_ddf = dd.read_parquet(write_path+"2005_pfl_temp.pq")

CPU times: user 50.3 ms, sys: 31 ms, total: 81.3 ms
Wall time: 76.3 ms


In [6]:
%%time
temp_csv_ddf = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)  # 25 MB chunks

CPU times: user 75.2 ms, sys: 9.09 ms, total: 84.3 ms
Wall time: 75.4 ms


In [9]:
%%time
temp_csv_ddf[temp_csv_ddf['z'] >= 2000].compute()

CPU times: user 8.09 s, sys: 836 ms, total: 8.92 s
Wall time: 40.9 s


,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
14071,15,1,27.490,10405576,2005-01-01 03:17:59.999827968,-13.809,81.542,b'',b'UNITED STATES',b'profiling float',1959.0,b'GTSP Program',b'U.S. ARGO PROJECT',b'US028807',b'UNIVERSITY OF WASHINGTON; SEATTLE',"b'APEX (Autonomous Profiling Explorer, Webb Re...","b'CTD: Sea-Bird Electronics, MODEL UNKNOWN'",3066.0000,b'WODpfl200501010015'
16436,96,71,4.567,10405654,2005-01-01 14:31:51.960982528,36.483,-14.506,b'ARGO profiling floats',b'SPAIN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'ES001259',b'',"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',2005.9418,b'WODpfl200501010096'
53666,212,55,2.949,10405750,2005-01-02 08:00:57.238769664,-26.886,-26.415,b'ARGO profiling floats',b'GREAT BRITAIN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'ARGO UK',b'GB011482',b'',"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',2003.7826,b'WODpfl200501020070'
26382,653,71,3.753,10406204,2005-01-05 10:30:00.000000000,44.233,-9.640,b'ARGO profiling floats',b'SPAIN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'ES001250',b'',"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',2004.0397,b'WODpfl200501050062'
1040,939,0,21.430,10406486,2005-01-07 06:28:00.000113664,-16.613,-116.599,b'',b'UNITED STATES',b'profiling float',1970.0,b'GTSP Program',b'U.S. ARGO PROJECT',b'US028886',b'UNIVERSITY OF WASHINGTON; SEATTLE',"b'APEX (Autonomous Profiling Explorer, Webb Re...","b'CTD: Sea-Bird Electronics, MODEL UNKNOWN'",3081.0000,b'WODpfl200501070050'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45354,63133,55,2.821,10624623,2005-12-27 08:12:11.160735744,-29.524,-37.417,b'ARGO profiling floats',b'GREAT BRITAIN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'ARGO UK',b'GB011473',b'',"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',2004.0653,b'WODpfl200512270056'
76073,63241,91,3.430,10624770,2005-12-28 00:50:38.039874560,48.310,-14.918,b'ARGO profiling floats',b'FRANCE',b'profiling float',42682.0,b'US GODAE server (Argo)',b'CONGAS (CONTINENTAL GASCOGNE)',b'FR014222',b'',"b'PROVOR (free-drifting hydrographic profiler,...",b'CTD: TYPE UNKNOWN',2018.9020,b'WODpfl200512280005'
66438,63309,122,3.516,10624836,2005-12-28 07:04:41.160735744,3.241,-13.970,b'ARGO profiling floats',b'FRANCE',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'FR014496',b'',"b'PROVOR (free-drifting hydrographic profiler,...",b'CTD: TYPE UNKNOWN',2004.1403,b'WODpfl200512280073'
12471,63957,122,3.460,10625530,2005-12-31 07:21:32.761230336,-5.114,-9.137,b'ARGO profiling floats',b'FRANCE',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'FR014490',b'',"b'PROVOR (free-drifting hydrographic profiler,...",b'CTD: TYPE UNKNOWN',2010.0000,b'WODpfl200512310064'


In [10]:
%%time
temp_pq_ddf[temp_pq_ddf['z'] >= 2000].compute()

CPU times: user 1.72 s, sys: 107 ms, total: 1.83 s
Wall time: 3.35 s


,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
14071,15,1,27.490,10405576,2005-01-01 03:17:59.999827968,-13.809000,81.542000,,UNITED STATES,profiling float,1959.0,GTSP Program,U.S. ARGO PROJECT,US028807,UNIVERSITY OF WASHINGTON; SEATTLE,"APEX (Autonomous Profiling Explorer, Webb Rese...","CTD: Sea-Bird Electronics, MODEL UNKNOWN",3066.000000,WODpfl200501010015
95546,96,71,4.567,10405654,2005-01-01 14:31:51.960982528,36.483002,-14.506000,ARGO profiling floats,SPAIN,profiling float,42682.0,US GODAE server (Argo),,ES001259,,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,2005.941772,WODpfl200501010096
212110,212,55,2.949,10405750,2005-01-02 08:00:57.238769664,-26.886000,-26.415001,ARGO profiling floats,GREAT BRITAIN,profiling float,42682.0,US GODAE server (Argo),ARGO UK,GB011482,,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,2003.782593,WODpfl200501020070
655331,653,71,3.753,10406204,2005-01-05 10:30:00.000000000,44.233002,-9.640000,ARGO profiling floats,SPAIN,profiling float,42682.0,US GODAE server (Argo),,ES001250,,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,2004.039673,WODpfl200501050062
942690,939,0,21.430,10406486,2005-01-07 06:28:00.000113664,-16.613001,-116.598999,,UNITED STATES,profiling float,1970.0,GTSP Program,U.S. ARGO PROJECT,US028886,UNIVERSITY OF WASHINGTON; SEATTLE,"APEX (Autonomous Profiling Explorer, Webb Rese...","CTD: Sea-Bird Electronics, MODEL UNKNOWN",3081.000000,WODpfl200501070050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63447715,63133,55,2.821,10624623,2005-12-27 08:12:11.160735744,-29.524000,-37.417000,ARGO profiling floats,GREAT BRITAIN,profiling float,42682.0,US GODAE server (Argo),ARGO UK,GB011473,,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,2004.065308,WODpfl200512270056
63556291,63241,91,3.430,10624770,2005-12-28 00:50:38.039874560,48.310001,-14.918000,ARGO profiling floats,FRANCE,profiling float,42682.0,US GODAE server (Argo),CONGAS (CONTINENTAL GASCOGNE),FR014222,,"PROVOR (free-drifting hydrographic profiler, I...",CTD: TYPE UNKNOWN,2018.901978,WODpfl200512280005
63624662,63309,122,3.516,10624836,2005-12-28 07:04:41.160735744,3.241000,-13.970000,ARGO profiling floats,FRANCE,profiling float,42682.0,US GODAE server (Argo),,FR014496,,"PROVOR (free-drifting hydrographic profiler, I...",CTD: TYPE UNKNOWN,2004.140259,WODpfl200512280073
64275902,63957,122,3.460,10625530,2005-12-31 07:21:32.761230336,-5.114000,-9.137000,ARGO profiling floats,FRANCE,profiling float,42682.0,US GODAE server (Argo),,FR014490,,"PROVOR (free-drifting hydrographic profiler, I...",CTD: TYPE UNKNOWN,2010.000000,WODpfl200512310064


# list of todo's

- create parquet from a df
- consider utility of turning "metadata" vars into coords
- consider removing NaN's at df step
- consider removing NaN's otherwise



In [ ]:
df.dropna(how='all',column='z')

In [ ]:
%%time
df = ds.to_dataframe()#.reset_index()